In [1]:
import numpy as np
!pip install ucimlrepo


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
covertype = fetch_ucirepo(id=31)

# data (as pandas dataframes)
X = covertype.data.features
y = covertype.data.targets

In [2]:
X

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,0
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,0
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,0
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,0
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,...,0,0,0,0,0,0,0,0,1,0
581008,2391,152,19,67,12,95,240,237,119,845,...,0,0,0,0,0,0,0,0,1,0
581009,2386,159,17,60,7,90,236,241,130,854,...,0,0,0,0,0,0,0,0,1,0
581010,2384,170,15,60,5,90,230,245,143,864,...,0,0,0,0,0,0,0,0,1,0


In [3]:
(y.Cover_Type.unique())

array([5, 2, 1, 7, 3, 6, 4])

In [4]:
y.nunique()

Cover_Type    7
dtype: int64

In [10]:
from src.active import ActiveModule
from src.strategies import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
np.random.seed(42)
initial_idx = np.random.randint(low=0, high=X_train.shape[0], size=int(0.01*X_train.shape[0]))

In [13]:
learner = ActiveModule(estimator=KNeighborsClassifier,
                       X=X_train.values,
                       y_initial=y_train.values[initial_idx],
                       label_idx=initial_idx,
                       X_valid=X_test.values,
                       y_valid=y_test.values,
                       n_neighbors=7)

In [14]:
initial_idx.shape

(4648,)

In [ ]:
_y, q_idx = None, None
for i in tqdm(range(10)):
    q_idx, _ = learner.step(_y, q_idx, strategy=random_sampling, n_samples=1000)
    _y = y_train.values[q_idx]

 20%|██        | 2/10 [02:19<09:32, 71.62s/it]

In [ ]:
metrics = learner.get_metric_history()

metrics[["accuracy", "pr_auc", "roc_auc"]].plot()
plt.show()

In [9]:
learner.get_metric_history()

,default_metric,accuracy,precision,recall,f1,roc_auc,pr_auc,confusion_matrix
0,0.851734,0.851734,0.851734,0.851734,0.851734,0.836077,0.602643,"[[36009, 6150, 14, 0, 62, 9, 313], [4960, 5061..."
1,0.851854,0.851854,0.851854,0.851854,0.851854,0.835957,0.602776,"[[36010, 6155, 14, 0, 62, 9, 307], [4947, 5063..."
